#**Reconhecimento de Padrões**

*Trabalho 3: Classificadores Bayesianos, Critério MAP e LDA*

*- Naive Bayes*

*- Renoir Sampaio (389113)*

##**Preparação dos dados**


In [1]:
# Importando bibliotecas
import numpy as np # Algebra Linear
from numpy.linalg import inv, det # Inversa e determinante
import pandas as pd # Processamento de dados
import seaborn as sns # Plotagem
import matplotlib.pyplot as plt # Plotagem
import math # Matemática
from random import randint # Aleatoriedade
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Realiza a leitura do dataset
df = pd.read_fwf('/content/data_dermato_03.txt', sep = '  ',
                 names = [' 1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
                          '11', '12', '13', '14', '15', '16', '17', '18', '19',
                          '20', '21', '22', '23', '24', '25', '26', '27', '28',
                          '29', '30', '31', '32', '33', '34', 'classe'])

In [3]:
# Dimensão do dataset
df.shape

(366, 35)

In [4]:
# Mostrando o dataset
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,classe
0,3,3,3,2,1,0,0,0,1,1,1,0,0,1,0,1,2,0,2,2,2,2,2,1,0,0,0,0,0,0,0,1,0,8,1
1,2,2,2,0,0,0,0,0,3,2,0,0,0,3,0,0,2,0,3,2,2,2,2,0,0,3,0,0,0,0,0,3,0,40,1
2,3,3,2,1,1,0,0,0,2,2,1,0,0,0,0,0,3,2,3,2,2,2,1,1,0,0,0,0,0,0,0,1,0,20,1
3,2,3,3,0,0,0,0,0,1,1,1,0,0,1,0,0,2,1,2,1,2,3,0,2,0,0,0,0,0,0,0,2,0,10,1
4,3,3,3,0,0,0,0,0,3,3,1,0,0,2,0,0,2,0,2,3,3,3,2,3,0,3,0,0,0,0,0,2,0,38,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,2,2,2,0,0,0,1,0,1,0,1,0,0,0,0,2,2,1,0,0,0,0,0,0,0,0,0,1,0,3,2,2,0,9,6
362,2,2,1,0,0,0,2,0,2,0,0,0,0,0,0,3,2,0,1,0,0,0,0,0,0,0,0,2,0,2,3,1,0,16,6
363,2,2,0,1,0,0,2,0,2,0,0,0,0,0,0,2,2,0,0,0,0,0,0,0,0,0,0,1,0,2,2,2,0,22,6
364,3,2,2,0,0,0,3,0,1,0,1,0,0,0,0,1,3,1,1,0,0,0,0,0,0,0,0,2,0,3,2,2,0,10,6


In [5]:
# Função divisão treino e teste
def divTesteTreinoTransp(df, percent, lengh):
  i = lengh*percent/100
  return df[0:int(i)], df[int(i):lengh]

In [6]:
# Função para calcular probabilidade a priori
def probApriori(total, classe):
  prob_apriori = classe/total
  return prob_apriori

In [7]:
# Matriz de covariância
def matrizCov(classTransp, numAtrib, N):
  COV = []
  for i in range(numAtrib):
    COV_prov = [] 
    for j in range(numAtrib):
      sum = 0
      for k in range(classTransp.shape[1]):
        xk = classTransp.T.values[k]
        sum = sum + (xk[i] - vetor_mean_general.values[i]) * (xk[j] - vetor_mean_general.values[j])
      if i == j:
        COV_prov.append(sum[0]/N + 0.01)
      else:
        COV_prov.append(sum[0]/N)
    COV.append(COV_prov)

  return COV

In [9]:
# Criação de função para cálculo de acurácia baseado na coluna 'classe'  
# da tabela dados gerais, onde estão localizados os labels
def getAccuracy(testSet, predictions, tamanho):
    correct = 0
    for x in range(len(testSet)):
        if (testSet[x] == predictions[x]):
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [10]:
# Função para criar matriz descorrelacionada
def matriz_desc(matriz):
  S1_Descorr = matriz
  for i in range(len(matriz)):
    for j in range(len(matriz)):
      if i == j:
        S1_Descorr[i][j] = S1_Descorr[i][j]
      else:
        S1_Descorr[i][j] = 0
  
  return S1_Descorr

In [11]:
# Função de normalização z-score
def normal_zscore(df):
  tam = len(df.T)-1
  df_zscore = (df.iloc[:,0:tam] - df.iloc[:,0:tam].mean()/df.iloc[:,0:tam].std())
  df_zscore['classe'] = df['classe'].values
  return df_zscore

In [76]:
# Função para cálculo da matriz de correlação
def matrizCorr(matrix, vetmeangeral_transp):
  CORR = pd.DataFrame(index = [matrix.columns[i] for i in range(matrix.shape[1]-1)], columns = matrix.columns[:])
  for i, conti in zip(matrix.columns[:-1], [i for i in range(matrix.shape[1])]):
    for j, contj in zip(matrix.columns[:-1], [i for i in range(matrix.shape[1])]):

       sum = np.sum(np.dot((matrix[i] - vetmeangeral_transp[i].values), (matrix[j] - vetmeangeral_transp[j].values)))
       denominador = np.sqrt(np.dot(np.sum(pow(matrix[i] - vetmeangeral_transp[i].values, 2)), np.sum(pow(matrix[j] - vetmeangeral_transp[j].values, 2))))
       CORR.iloc[conti, contj] = sum/denominador
       
  return CORR

## ***Naive Bayes:***

In [12]:
# K-Fold com as 5 interações
all_accuracy = []
precision_scores = []
for p in range(5):
  # Normalização zscore
  df = normal_zscore(df)
  
  # Criar dataset sem classes e aleatório
  df_sample = df.sample(frac=1)
  
  # Divisão treino e teste 70% e 30%
  treino, teste = divTesteTreinoTransp(df_sample, 70, len(df))

  # Retirando classe dos datasets
  treino_noclass = treino.drop(columns = ['classe'])
  teste_noclass = teste.drop(columns = ['classe'])

  # Selecionando classes do dataset de treino
  df_sample_class1 = treino.loc[treino['classe'] == 1]
  df_sample_class2 = treino.loc[treino['classe'] == 2]
  df_sample_class3 = treino.loc[treino['classe'] == 3]
  df_sample_class4 = treino.loc[treino['classe'] == 4]
  df_sample_class5 = treino.loc[treino['classe'] == 5]
  df_sample_class6 = treino.loc[treino['classe'] == 6]

  # Retirando coluna 'classe' dos dados de treino
  df_sample_noclass_class1 = df_sample_class1.drop(columns = ['classe'])
  df_sample_noclass_class2 = df_sample_class2.drop(columns = ['classe'])
  df_sample_noclass_class3 = df_sample_class3.drop(columns = ['classe'])
  df_sample_noclass_class4 = df_sample_class4.drop(columns = ['classe'])
  df_sample_noclass_class5 = df_sample_class5.drop(columns = ['classe'])
  df_sample_noclass_class6 = df_sample_class6.drop(columns = ['classe'])

  # Calcular a probabilidade a priori de cada classe
  probApriori_classe1 = probApriori(len(treino_noclass), len(df_sample_noclass_class1))
  probApriori_classe2 = probApriori(len(treino_noclass), len(df_sample_noclass_class2))
  probApriori_classe3 = probApriori(len(treino_noclass), len(df_sample_noclass_class3))
  probApriori_classe4 = probApriori(len(treino_noclass), len(df_sample_noclass_class4))
  probApriori_classe5 = probApriori(len(treino_noclass), len(df_sample_noclass_class5))
  probApriori_classe6 = probApriori(len(treino_noclass), len(df_sample_noclass_class6))

  # Transposição dos datasets
  treino_noclass_transp = treino_noclass.T
  teste_noclass_transp = teste_noclass.T
  df_sample_noclass_class1_transp = df_sample_noclass_class1.T
  df_sample_noclass_class2_transp = df_sample_noclass_class2.T
  df_sample_noclass_class3_transp = df_sample_noclass_class3.T
  df_sample_noclass_class4_transp = df_sample_noclass_class4.T
  df_sample_noclass_class5_transp = df_sample_noclass_class6.T
  df_sample_noclass_class6_transp = df_sample_noclass_class3.T

  # Cálculo vetor média dos dados gerais
  vetor_mean_general = treino_noclass.mean()
  vetor_mean_general = pd.DataFrame(vetor_mean_general)

  # Sw matrix de covariância intra classes (Soma das matrizes de covariância de
  # muitas classes - somatório de Si)
  S1 = matrizCov(df_sample_noclass_class1_transp, len(df_sample_noclass_class1_transp), len(df_sample_noclass_class1))
  S1 = pd.DataFrame(S1)
  S2 = matrizCov(df_sample_noclass_class2_transp, len(df_sample_noclass_class2_transp), len(df_sample_noclass_class2))
  S2 = pd.DataFrame(S2)
  S3 = matrizCov(df_sample_noclass_class3_transp, len(df_sample_noclass_class3_transp), len(df_sample_noclass_class3))
  S3 = pd.DataFrame(S3)
  S4 = matrizCov(df_sample_noclass_class4_transp, len(df_sample_noclass_class4_transp), len(df_sample_noclass_class4))
  S4 = pd.DataFrame(S4)
  S5 = matrizCov(df_sample_noclass_class5_transp, len(df_sample_noclass_class5_transp), len(df_sample_noclass_class5))
  S5 = pd.DataFrame(S5)
  S6 = matrizCov(df_sample_noclass_class6_transp, len(df_sample_noclass_class6_transp), len(df_sample_noclass_class6))
  S6 = pd.DataFrame(S6)
  Sw = S1 + S2 + S3 + S4 + S5 + S6

  # Quantidade de elementos de cada classe
  qtd1 = len(df_sample_noclass_class1)
  qtd2 = len(df_sample_noclass_class2)
  qtd3 = len(df_sample_noclass_class3)
  qtd4 = len(df_sample_noclass_class4)
  qtd5 = len(df_sample_noclass_class5)
  qtd6 = len(df_sample_noclass_class6)

  # Variância dos atributos de todos os dados e de cada classe
  VaT = np.var(treino_noclass)
  VaT = pd.DataFrame(VaT)
  Va1 = np.var(df_sample_noclass_class1)
  Va1 = pd.DataFrame(Va1)
  Va2 = np.var(df_sample_noclass_class2)
  Va2 = pd.DataFrame(Va2)
  Va3 = np.var(df_sample_noclass_class3)
  Va3 = pd.DataFrame(Va3)
  Va4 = np.var(df_sample_noclass_class4)
  Va4 = pd.DataFrame(Va4)
  Va5 = np.var(df_sample_noclass_class5)
  Va5 = pd.DataFrame(Va5)
  Va6 = np.var(df_sample_noclass_class6)
  Va6 = pd.DataFrame(Va6)

  # Vetor média das classes
  Mu1 = df_sample_noclass_class1.mean()
  Mu1 = pd.DataFrame(Mu1)
  Mu2 = df_sample_noclass_class2.mean()
  Mu2 = pd.DataFrame(Mu2)
  Mu3 = df_sample_noclass_class3.mean()
  Mu3 = pd.DataFrame(Mu3)
  Mu4 = df_sample_noclass_class4.mean()
  Mu4 = pd.DataFrame(Mu4)
  Mu5 = df_sample_noclass_class5.mean()
  Mu5 = pd.DataFrame(Mu5)
  Mu6 = df_sample_noclass_class6.mean()
  Mu6 = pd.DataFrame(Mu6)

  # SB matrix de espalhamento entre classes
  SB1 = pd.DataFrame(qtd1*(vetor_mean_general - Mu1).values*((vetor_mean_general - Mu1).T).values)
  SB2 = pd.DataFrame(qtd2*(vetor_mean_general - Mu2).values*((vetor_mean_general - Mu2).T).values)
  SB3 = pd.DataFrame(qtd3*(vetor_mean_general - Mu3).values*((vetor_mean_general - Mu3).T).values)
  SB4 = pd.DataFrame(qtd4*(vetor_mean_general - Mu4).values*((vetor_mean_general - Mu4).T).values)
  SB5 = pd.DataFrame(qtd5*(vetor_mean_general - Mu5).values*((vetor_mean_general - Mu5).T).values)
  SB6 = pd.DataFrame(qtd6*(vetor_mean_general - Mu6).values*((vetor_mean_general - Mu6).T).values)
  SB = SB1 + SB2 + SB3 + SB4 + SB5 + SB6

  # Matrizes descorrelacionadas
  S1_d = matriz_desc(S1)
  S2_d = matriz_desc(S2)
  S3_d = matriz_desc(S3)
  S4_d = matriz_desc(S4)
  S5_d = matriz_desc(S5)
  S6_d = matriz_desc(S6)

  # Cálculo Naive Bayes e escolha dos menores valores para definir índice 
  # (classe) a qual a amostra pertence escolhendo valor mínimo no vetor de
  # valore
  Bayes_indices = []
  labels = []
  for i in range(len(teste_noclass)):
    Bayes_vet = []
    X = teste_noclass.values[i, :]
    Bayes_prov1 = float(np.log(det(S1_d)) + (((X - Mu1.T).values @ inv(S1_d)) @ ((X - Mu1.T).T.values)) - (2 * np.log(probApriori_classe1)))
    Bayes_vet.append(Bayes_prov1)
    Bayes_prov2 = float(np.log(det(S2_d)) + (((X - Mu2.T).values @ inv(S2_d)) @ ((X - Mu2.T).T.values)) - (2 * np.log(probApriori_classe2)))
    Bayes_vet.append(Bayes_prov2)
    Bayes_prov3 = float(np.log(det(S3_d)) + (((X - Mu3.T).values @ inv(S3_d)) @ ((X - Mu3.T).T.values)) - (2 * np.log(probApriori_classe3)))
    Bayes_vet.append(Bayes_prov3)
    Bayes_prov4 = float(np.log(det(S4_d)) + (((X - Mu4.T).values @ inv(S4_d)) @ ((X - Mu4.T).T.values)) - (2 * np.log(probApriori_classe4)))
    Bayes_vet.append(Bayes_prov4)
    Bayes_prov5 = float(np.log(det(S5_d)) + (((X - Mu5.T).values @ inv(S5_d)) @ ((X - Mu5.T).T.values)) - (2 * np.log(probApriori_classe5)))
    Bayes_vet.append(Bayes_prov5)
    Bayes_prov6 = float(np.log(det(S6_d)) + (((X - Mu6.T).values @ inv(S6_d)) @ ((X - Mu6.T).T.values)) - (2 * np.log(probApriori_classe6)))
    Bayes_vet.append(Bayes_prov6)
    Bayes_indices.append(Bayes_vet.index(np.min(Bayes_vet))+1)

  # Vetores de índices para comparação
  Bayes_indices = np.array(Bayes_indices)
  indices_teste = teste['classe'].values

  # Teste de acurácia com labels corretos e labels previstos (classificados)
  accuracy = getAccuracy(indices_teste, Bayes_indices, len(indices_teste))
  all_accuracy.append(accuracy)

  # Teste de precisão com labels corretos e labels previstos (classificados)
  precision = round(precision_score(indices_teste, Bayes_indices, average = 'weighted'), 6)
  precision_scores.append(precision)

# Média total das acurácias
print(f'Média das acurácias: {int(np.mean(all_accuracy))}%')

# Média total das precisões
print(f'Média das precisão: {int(np.mean(precision_scores)*100)}%')

Média das acurácias: 92%
Média das precisão: 93%


In [13]:
# Matriz de confusão para visualização dos erros e acertos
print(confusion_matrix(indices_teste, Bayes_indices))

[[32  2  0  0  0  0]
 [ 0 13  0  3  0  0]
 [ 0  0 23  0  0  0]
 [ 0  0  0 11  0  0]
 [ 0  1  0  2 15  0]
 [ 0  0  0  0  0  8]]


In [14]:
# Classificações em mais detalhes
print(classification_report(indices_teste, Bayes_indices))

              precision    recall  f1-score   support

           1       1.00      0.94      0.97        34
           2       0.81      0.81      0.81        16
           3       1.00      1.00      1.00        23
           4       0.69      1.00      0.81        11
           5       1.00      0.83      0.91        18
           6       1.00      1.00      1.00         8

    accuracy                           0.93       110
   macro avg       0.92      0.93      0.92       110
weighted avg       0.94      0.93      0.93       110



In [ ]:
# Matriz de correlação dos dados 
Corr_Matrix = matrizCorr(treino_noclass, vetor_mean_general.T)
Corr_Matrix